<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/00_CVAT_Ground_Truth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
CONVERSIÓN DE ANOTACIONES CVAT A MÁSCARAS GROUND TRUTH NPZ

Trabajo Fin de Máster: Evaluación Comparativa de Técnicas de Segmentación
en Fotografía de Retrato

Autor: Jesús L.
Institución: Universidad Oberta de Cataluña (UOC)
Fecha: Noviembre 2025

DESCRIPCIÓN:
Este módulo implementa la conversión de anotaciones manuales realizadas en CVAT
(Computer Vision Annotation Tool) al formato de máscaras binarias NPZ utilizado
en el sistema de evaluación del TFM. Las anotaciones en formato COCO se
transforman en máscaras de segmentación compatibles con los evaluadores
desarrollados para SAM 2.0, YOLOv8-seg, Mask2Former, OneFormer y BodyPix.

OBJETIVO:
Generar máscaras ground truth de alta calidad para la evaluación cuantitativa
objetiva de los modelos de segmentación implementados, calculando métricas
como IoU (Intersection over Union), Precision, Recall y F1-Score.

PROCESO:
1. Montaje de Google Drive y acceso al directorio del proyecto
2. Carga y validación de anotaciones COCO exportadas desde CVAT
3. Conversión de polígonos de segmentación a máscaras binarias
4. Validación de calidad de las anotaciones generadas
5. Guardado en formato NPZ compatible con los evaluadores existentes

ESTRUCTURA DE DATOS:

Entrada (COCO format):
    - instances_default.json: Anotaciones exportadas desde CVAT
    - Contiene: imágenes, anotaciones (polígonos), categorías
    - Ubicación: /content/drive/MyDrive/TFM/0_Imagenes_CVAT/

Salida (NPZ format):
    - {imagen_nombre}_gt.npz: Máscara ground truth por imagen
    - Estructura:
        * masks: np.ndarray [H, W] normalizado [0, 1]
        * scores: np.array([1.0]) - confianza 100% (ground truth)
        * metadata: JSON con información de la anotación
    - Ubicación: /content/drive/MyDrive/TFM/0_Imagenes_CVAT/ground_truth_masks/

METODOLOGÍA:
La conversión de polígonos a máscaras binarias se realiza mediante rasterización
utilizando PIL.ImageDraw. Para imágenes con múltiples personas, las máscaras
individuales se combinan mediante operación OR lógica, generando una máscara
unificada que representa todas las instancias de personas en la imagen.

VALIDACIÓN DE CALIDAD:
Se implementan verificaciones automáticas de calidad que evalúan:
    - Cobertura de la máscara (porcentaje de píxeles anotados)
    - Número de regiones conectadas (detección de fragmentación)
    - Presencia de huecos internos en la máscara
    - Validez general basada en umbrales empíricos para fotografía de retrato

COMPATIBILIDAD:
El formato de salida NPZ es compatible con el sistema de checkpointing
implementado en todos los evaluadores del TFM, permitiendo integración
directa con el pipeline de análisis cuantitativo desarrollado en el Notebook 3.

REFERENCIAS:
- COCO format: https://cocodataset.org/#format-data
- CVAT documentation: https://opencv.github.io/cvat/
- Shapely geometry: https://shapely.readthedocs.io/
"""

'\nCONVERSIÓN DE ANOTACIONES CVAT A MÁSCARAS GROUND TRUTH NPZ\n\nTrabajo Fin de Máster: Evaluación Comparativa de Técnicas de Segmentación \nen Fotografía de Retrato\n\nAutor: Jesús L.\nInstitución: Universidad Oberta de Cataluña (UOC)\nFecha: Noviembre 2025\n\nDESCRIPCIÓN:\nEste módulo implementa la conversión de anotaciones manuales realizadas en CVAT\n(Computer Vision Annotation Tool) al formato de máscaras binarias NPZ utilizado\nen el sistema de evaluación del TFM. Las anotaciones en formato COCO se \ntransforman en máscaras de segmentación compatibles con los evaluadores \ndesarrollados para SAM 2.0, YOLOv8-seg, Mask2Former, OneFormer y BodyPix.\n\nOBJETIVO:\nGenerar máscaras ground truth de alta calidad para la evaluación cuantitativa\nobjetiva de los modelos de segmentación implementados, calculando métricas\ncomo IoU (Intersection over Union), Precision, Recall y F1-Score.\n\nPROCESO:\n1. Montaje de Google Drive y acceso al directorio del proyecto\n2. Carga y validación de ano

In [2]:
import json
import numpy as np
from PIL import Image, ImageDraw
from pathlib import Path
from tqdm import tqdm
from typing import Dict, List, Tuple, Optional
import cv2
import warnings

# Suprimir warnings de deprecación para mantener output limpio
warnings.filterwarnings('ignore')

In [3]:
# ==============================================================================
# CONFIGURACIÓN Y MONTAJE DE GOOGLE DRIVE
# ==============================================================================

def setup_google_drive():
    """
    Configura el acceso a Google Drive desde Google Colab.

    Monta el sistema de archivos de Google Drive en /content/drive/
    permitiendo acceso directo a los archivos del proyecto TFM.

    Returns:
        Path: Ruta base al directorio del proyecto en Google Drive

    Raises:
        RuntimeError: Si no se está ejecutando en Google Colab
    """
    try:
        from google.colab import drive
        print("Montando Google Drive...")
        drive.mount('/content/drive', force_remount=False)

        base_path = Path('/content/drive/MyDrive/TFM/0_Imagenes_CVAT')

        if not base_path.exists():
            raise FileNotFoundError(
                f"No se encuentra el directorio del proyecto: {base_path}\n"
                "Verifique que la estructura de carpetas sea correcta."
            )

        print(f"Google Drive montado correctamente.")
        print(f"Directorio del proyecto: {base_path}")
        return base_path

    except ImportError:
        raise RuntimeError(
            "Este script está diseñado para ejecutarse en Google Colab.\n"
            "Para ejecución local, modifique la función setup_google_drive()."
        )

In [4]:
# ==============================================================================
# CONVERSIÓN DE GEOMETRÍAS
# ==============================================================================

def polygon_to_mask(polygon: List[float],
                    image_size: Tuple[int, int]) -> np.ndarray:
    """
    Convierte un polígono en formato COCO a máscara binaria mediante rasterización.

    El formato COCO representa polígonos como listas planas de coordenadas
    [x1, y1, x2, y2, ..., xn, yn]. Esta función convierte estas coordenadas
    en una máscara binaria del tamaño especificado.

    Args:
        polygon: Lista de coordenadas en formato COCO [x1,y1,x2,y2,...]
        image_size: Tupla (width, height) del tamaño de la imagen destino

    Returns:
        Máscara binaria como numpy array de shape (height, width) con valores
        0 (fondo) y 255 (persona)

    Notes:
        - Se requieren mínimo 3 puntos para formar un polígono válido
        - Utiliza PIL.ImageDraw para rasterización eficiente
        - El polígono se rellena completamente (no solo el contorno)
    """
    # Crear imagen vacía para la máscara
    mask = Image.new('L', image_size, 0)
    draw = ImageDraw.Draw(mask)

    # Convertir lista plana a lista de tuplas (x, y)
    points = [(polygon[i], polygon[i+1]) for i in range(0, len(polygon), 2)]

    # Validar número mínimo de puntos para polígono válido
    if len(points) >= 3:
        draw.polygon(points, fill=255)
    else:
        warnings.warn(
            f"Polígono con {len(points)} puntos ignorado (mínimo 3 requeridos)"
        )

    return np.array(mask)


def rle_to_mask(rle: Dict, image_size: Tuple[int, int]) -> np.ndarray:
    """
    Convierte Run Length Encoding (RLE) a máscara binaria.

    Aunque CVAT típicamente exporta polígonos, algunos formatos o
    configuraciones pueden utilizar RLE para compresión eficiente.
    Esta función proporciona compatibilidad con ambos formatos.

    Args:
        rle: Diccionario con formato RLE de COCO ('counts' y 'size')
        image_size: Tupla (width, height) del tamaño de imagen

    Returns:
        Máscara binaria como numpy array

    Requires:
        pycocotools (instalado via: pip install pycocotools)

    Notes:
        RLE es más eficiente para almacenamiento pero menos común en
        exportaciones estándar de CVAT para tareas de segmentación manual.
    """
    try:
        from pycocotools import mask as mask_utils

        if isinstance(rle, dict):
            mask = mask_utils.decode(rle)
        else:
            # RLE compacto (formato alternativo)
            mask = mask_utils.decode({
                'counts': rle,
                'size': image_size[::-1]
            })

        return mask

    except ImportError:
        raise ImportError(
            "pycocotools no está instalado. Ejecute:\n"
            "pip install pycocotools"
        )

In [5]:
# ==============================================================================
# VALIDACIÓN DE CALIDAD
# ==============================================================================

def validate_mask_quality(mask: np.ndarray,
                         image_size: Tuple[int, int],
                         image_name: str = "unknown") -> Dict:
    """
    Valida la calidad de una máscara ground truth mediante análisis geométrico.

    Implementa heurísticas de validación basadas en características esperadas
    de fotografía de retrato. Detecta potenciales problemas de anotación como
    fragmentación excesiva, cobertura anómala o presencia de huecos internos.

    Args:
        mask: Máscara binaria a validar (valores 0-255)
        image_size: Tupla (width, height) de la imagen original
        image_name: Nombre de la imagen para reporting

    Returns:
        Diccionario con métricas de calidad:
            - coverage_percent: Porcentaje de píxeles anotados
            - num_regions: Número de componentes conectados (personas separadas)
            - has_holes: Presencia de huecos internos en las máscaras
            - is_valid: Validación global basada en umbrales empíricos
            - warnings: Lista de advertencias detectadas

    Notes:
        Umbrales de validación basados en análisis empírico de fotografía
        de retrato típica:
            - Cobertura: 10-90% (persona visible sin dominar completamente)
            - Regiones: 1-3 (retratos individuales o grupales pequeños)
            - Huecos: Aceptables pero reportados para revisión manual
    """
    total_pixels = image_size[0] * image_size[1]
    mask_pixels = np.sum(mask > 0)
    coverage = mask_pixels / total_pixels

    # Detectar componentes conectados mediante análisis de conectividad
    # Utiliza conectividad-8 (incluye diagonales) apropiada para segmentación
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(
        mask.astype(np.uint8),
        connectivity=8
    )

    # Número de regiones excluyendo el fondo (label 0)
    num_regions = num_labels - 1

    # Detectar huecos internos mediante análisis de contornos jerárquico
    # RETR_CCOMP recupera jerarquía de dos niveles: externos e internos
    contours, hierarchy = cv2.findContours(
        mask.astype(np.uint8),
        cv2.RETR_CCOMP,
        cv2.CHAIN_APPROX_SIMPLE
    )

    # Huecos presentes si existen contornos internos (hijos en jerarquía)
    has_holes = (
        hierarchy is not None and
        np.any(hierarchy[0, :, 3] != -1)
    )

    # Compilar advertencias basadas en análisis
    warnings_list = []

    if coverage < 0.05:
        warnings_list.append("Cobertura muy baja (<5%). Verificar anotación.")
    elif coverage > 0.95:
        warnings_list.append("Cobertura muy alta (>95%). Posible error.")

    if num_regions > 5:
        warnings_list.append(
            f"Fragmentación alta ({num_regions} regiones). "
            "Revisar continuidad de la anotación."
        )

    if has_holes:
        warnings_list.append(
            "Huecos internos detectados. Verificar si es intencional "
            "(ej. brazos formando hueco)."
        )

    # Validación global basada en umbrales empíricos
    # Cobertura razonable para retratos: entre 10% y 90%
    is_valid = 0.10 < coverage < 0.90

    return {
        'image_name': image_name,
        'coverage_percent': coverage * 100,
        'num_regions': num_regions,
        'has_holes': has_holes,
        'is_valid': is_valid,
        'warnings': warnings_list
    }

In [7]:
# ==============================================================================
# CONVERSIÓN PRINCIPAL
# ==============================================================================

def cvat_coco_to_ground_truth(coco_json_path: Path,
                               output_dir: Path,
                               validate: bool = True,
                               verbose: bool = True) -> Dict:
    """
    Convierte exportación COCO de CVAT a máscaras NPZ ground truth.

    Función principal que orquesta el proceso completo de conversión desde
    anotaciones en formato COCO (exportadas de CVAT) hasta máscaras binarias
    en formato NPZ compatible con el sistema de evaluación del TFM.

    Args:
        coco_json_path: Ruta al archivo instances_default.json de CVAT
        output_dir: Directorio donde guardar las máscaras NPZ generadas
        validate: Si True, ejecuta validación de calidad de anotaciones
        verbose: Si True, muestra información detallada del proceso

    Returns:
        Diccionario con estadísticas del proceso:
            - total_images: Número de imágenes procesadas
            - total_annotations: Número total de personas anotadas
            - images_single_person: Imágenes con una sola persona
            - images_multiple_persons: Imágenes con múltiples personas
            - quality_warnings: Lista de advertencias de calidad detectadas
            - processing_time: Tiempo total de procesamiento

    Raises:
        FileNotFoundError: Si no se encuentra el archivo COCO
        ValueError: Si el formato COCO es inválido

    Notes:
        - Soporta múltiples personas por imagen (máscaras combinadas)
        - Mantiene compatibilidad con formato NPZ de evaluadores existentes
        - Genera metadata completa para trazabilidad
        - Implementa checkpoint automático (guarda progresivamente)

    """
    import time
    start_time = time.time()

    # Validar existencia del archivo de entrada
    if not coco_json_path.exists():
        raise FileNotFoundError(
            f"No se encuentra el archivo de anotaciones COCO: {coco_json_path}"
        )

    if verbose:
        print("=" * 80)
        print("CONVERSIÓN DE ANOTACIONES CVAT A GROUND TRUTH")
        print("=" * 80)
        print(f"\nArchivo de entrada: {coco_json_path}")
        print(f"Directorio de salida: {output_dir}")
        print("\nCargando anotaciones COCO...")

    # Cargar archivo JSON con manejo de errores
    try:
        with open(coco_json_path, 'r', encoding='utf-8') as f:
            coco_data = json.load(f)
    except json.JSONDecodeError as e:
        raise ValueError(
            f"Error al parsear archivo COCO: {e}\n"
            "Verifique que el archivo JSON esté bien formado."
        )

    # Validar estructura básica del formato COCO
    required_keys = ['images', 'annotations', 'categories']
    missing_keys = [key for key in required_keys if key not in coco_data]
    if missing_keys:
        raise ValueError(
            f"Formato COCO inválido. Faltan claves: {missing_keys}"
        )

    # Construir índice de imágenes para acceso eficiente O(1)
    images_map = {img['id']: img for img in coco_data['images']}

    # Agrupar anotaciones por imagen
    # Estructura: {image_id: [annotation1, annotation2, ...]}
    annotations_by_image = {}
    for ann in coco_data['annotations']:
        img_id = ann['image_id']
        if img_id not in annotations_by_image:
            annotations_by_image[img_id] = []
        annotations_by_image[img_id].append(ann)

    # Crear directorio de salida si no existe
    output_dir.mkdir(parents=True, exist_ok=True)

    # Inicializar estadísticas del proceso
    stats = {
        'total_images': len(annotations_by_image),
        'total_annotations': sum(len(anns) for anns in annotations_by_image.values()),
        'images_single_person': 0,
        'images_multiple_persons': 0,
        'quality_warnings': [],
        'processing_time': 0
    }

    if verbose:
        print(f"\nDataset: {stats['total_images']} imágenes con "
              f"{stats['total_annotations']} anotaciones")
        print("\nGenerando máscaras ground truth...")
        print("-" * 80)

    # Procesar cada imagen con barra de progreso
    for img_id, annotations in tqdm(
        annotations_by_image.items(),
        disable=not verbose,
        desc="Procesando imágenes",
        unit="img"
    ):
        image_info = images_map[img_id]
        image_size = (image_info['width'], image_info['height'])
        filename_base = Path(image_info['file_name']).stem

        # Inicializar máscara combinada para todas las personas
        # Dtype uint8 para eficiencia de memoria
        combined_mask = np.zeros(image_size[::-1], dtype=np.uint8)

        # Contabilizar número de personas en esta imagen
        num_persons = len(annotations)
        if num_persons == 1:
            stats['images_single_person'] += 1
        else:
            stats['images_multiple_persons'] += 1

        # Procesar cada anotación (persona) en la imagen
        for ann in annotations:
            if 'segmentation' not in ann or not ann['segmentation']:
                warnings.warn(
                    f"Anotación sin segmentación en imagen {filename_base}, "
                    f"annotation_id: {ann.get('id', 'unknown')}"
                )
                continue

            # Procesar cada segmento (puede haber múltiples polígonos por persona)
            for seg in ann['segmentation']:
                person_mask = None

                if isinstance(seg, list):
                    # Formato polígono (más común en CVAT)
                    person_mask = polygon_to_mask(seg, image_size)
                elif isinstance(seg, dict):
                    # Formato RLE (menos común pero soportado)
                    person_mask = rle_to_mask(seg, image_size)
                else:
                    warnings.warn(
                        f"Formato de segmentación desconocido en {filename_base}: "
                        f"{type(seg)}"
                    )
                    continue

                if person_mask is not None:
                    # Combinar máscaras mediante OR lógico (unión de regiones)
                    # Esto permite múltiples personas en la misma imagen
                    combined_mask = np.maximum(combined_mask, person_mask)

        # Validar calidad de la máscara generada
        if validate and np.any(combined_mask):
            quality = validate_mask_quality(
                combined_mask,
                image_size,
                filename_base
            )

            # Registrar advertencias de calidad
            if quality['warnings']:
                for warning in quality['warnings']:
                    warning_msg = f"{filename_base}: {warning}"
                    stats['quality_warnings'].append(warning_msg)
                    if verbose:
                        print(f"\nAdvertencia: {warning_msg}")

            # Reportar métricas de calidad en modo verbose
            if verbose and not quality['is_valid']:
                print(f"\nAtención - {filename_base}:")
                print(f"  Cobertura: {quality['coverage_percent']:.2f}%")
                print(f"  Regiones: {quality['num_regions']}")
                print(f"  Huecos: {'Sí' if quality['has_holes'] else 'No'}")

        # Construir metadata completa para trazabilidad
        metadata = {
            'image_file': image_info['file_name'],
            'image_size': image_size,
            'annotation_source': 'cvat_manual_ground_truth',
            'num_persons': num_persons,
            'cvat_image_id': img_id,
            'annotator': 'jesus_alejandro',
            'project': 'TFM_Segmentacion_Retratos',
            'institution': 'Universidad Oberta de Cataluña (UOC)',
            'creation_date': time.strftime('%Y-%m-%d %H:%M:%S')
        }

        # Definir ruta de salida
        output_file = output_dir / f"{filename_base}_gt.npz"

        # Guardar en formato NPZ compatible con evaluadores
        np.savez_compressed(
            output_file,
            masks=combined_mask.astype(np.float32) / 255.0,  # Normalizar [0,1]
            scores=np.array([1.0]),  # Confianza 100% (ground truth)
            metadata=json.dumps(metadata)
        )

    # Calcular tiempo total de procesamiento
    stats['processing_time'] = time.time() - start_time

    # Generar reporte final
    if verbose:
        print("\n" + "=" * 80)
        print("CONVERSIÓN COMPLETADA")
        print("=" * 80)
        print(f"\nEstadísticas:")
        print(f"  Imágenes procesadas: {stats['total_images']}")
        print(f"  Total personas anotadas: {stats['total_annotations']}")
        print(f"  Promedio personas/imagen: "
              f"{stats['total_annotations']/stats['total_images']:.2f}")
        print(f"  Imágenes con 1 persona: {stats['images_single_person']}")
        print(f"  Imágenes con múltiples personas: "
              f"{stats['images_multiple_persons']}")
        print(f"  Tiempo de procesamiento: {stats['processing_time']:.2f} segundos")

        if stats['quality_warnings']:
            print(f"\nAdvertencias de calidad: {len(stats['quality_warnings'])}")
            print("Primeras 5 advertencias:")
            for warning in stats['quality_warnings'][:5]:
                print(f"  - {warning}")
            if len(stats['quality_warnings']) > 5:
                print(f"  ... y {len(stats['quality_warnings']) - 5} más")
        else:
            print("\nNo se detectaron advertencias de calidad.")

        print(f"\nMáscaras guardadas en: {output_dir}")
        print("=" * 80)

    return stats

In [8]:
# ==============================================================================
# FUNCIÓN PRINCIPAL DE EJECUCIÓN
# ==============================================================================

def main():
    """
    Función principal que ejecuta el pipeline completo de conversión.

    Orquesta el proceso completo:
    1. Montaje de Google Drive
    2. Validación de estructura de directorios
    3. Conversión de anotaciones COCO a máscaras NPZ
    4. Generación de reporte de resultados

    Raises:
        FileNotFoundError: Si no se encuentran los archivos o directorios
        RuntimeError: Si ocurre un error durante la conversión
    """

    try:
        # Configurar acceso a Google Drive
        base_path = setup_google_drive()

        # Definir rutas del proyecto
        coco_json_path = base_path / 'instances_Validation.json'
        output_dir = base_path / 'ground_truth_masks'

        print(f"\nRutas configuradas:")
        print(f"  Base: {base_path}")
        print(f"  Anotaciones COCO: {coco_json_path}")
        print(f"  Salida máscaras: {output_dir}")

        # Validar existencia del archivo de anotaciones
        if not coco_json_path.exists():
            raise FileNotFoundError(
                f"\nNo se encuentra el archivo de anotaciones COCO.\n"
            )

        print("\nValidación completada. Iniciando conversión...\n")

        # Ejecutar conversión
        stats = cvat_coco_to_ground_truth(
            coco_json_path=coco_json_path,
            output_dir=output_dir,
            validate=True,
            verbose=True
        )

        # Reporte final de éxito
        print("\n" + "=" * 80)
        print("PROCESO COMPLETADO EXITOSAMENTE")
        print("=" * 80)
        print(f"\nResumen de resultados:")
        print(f"  Total de máscaras ground truth generadas: {stats['total_images']}")
        print(f"  Ubicación: {output_dir}")
        print("=" * 80 + "\n")

        return stats

    except FileNotFoundError as e:
        print(f"\nError de archivo no encontrado:")
        print(str(e))
        raise

    except Exception as e:
        print(f"\nError durante la conversión:")
        print(f"  Tipo: {type(e).__name__}")
        print(f"  Mensaje: {str(e)}")
        import traceback
        print("\nTraceback completo:")
        traceback.print_exc()
        raise

In [9]:
# ==============================================================================
# PUNTO DE ENTRADA
# ==============================================================================

if __name__ == "__main__":
    results = main()

Montando Google Drive...
Mounted at /content/drive
Google Drive montado correctamente.
Directorio del proyecto: /content/drive/MyDrive/TFM/0_Imagenes_CVAT

Rutas configuradas:
  Base: /content/drive/MyDrive/TFM/0_Imagenes_CVAT
  Anotaciones COCO: /content/drive/MyDrive/TFM/0_Imagenes_CVAT/instances_Validation.json
  Salida máscaras: /content/drive/MyDrive/TFM/0_Imagenes_CVAT/ground_truth_masks

Validación completada. Iniciando conversión...

CONVERSIÓN DE ANOTACIONES CVAT A GROUND TRUTH

Archivo de entrada: /content/drive/MyDrive/TFM/0_Imagenes_CVAT/instances_Validation.json
Directorio de salida: /content/drive/MyDrive/TFM/0_Imagenes_CVAT/ground_truth_masks

Cargando anotaciones COCO...

Dataset: 20 imágenes con 20 anotaciones

Generando máscaras ground truth...
--------------------------------------------------------------------------------


Procesando imágenes:   5%|▌         | 1/20 [00:01<00:19,  1.04s/img]


Advertencia: _DSC0036: Huecos internos detectados. Verificar si es intencional (ej. brazos formando hueco).


Procesando imágenes:  15%|█▌        | 3/20 [00:02<00:15,  1.08img/s]


Advertencia: _DSC0084: Huecos internos detectados. Verificar si es intencional (ej. brazos formando hueco).


...


Advertencia: _DSC0441: Huecos internos detectados. Verificar si es intencional (ej. brazos formando hueco).


Procesando imágenes:  75%|███████▌  | 15/20 [00:14<00:04,  1.13img/s]


Advertencia: _DSC0584: Huecos internos detectados. Verificar si es intencional (ej. brazos formando hueco).


Procesando imágenes: 100%|██████████| 20/20 [00:19<00:00,  1.01img/s]


CONVERSIÓN COMPLETADA

Estadísticas:
  Imágenes procesadas: 20
  Total personas anotadas: 20
  Promedio personas/imagen: 1.00
  Imágenes con 1 persona: 20
  Imágenes con múltiples personas: 0
  Tiempo de procesamiento: 20.79 segundos

Advertencias de calidad: 8
Primeras 5 advertencias:
  - _DSC0036: Huecos internos detectados. Verificar si es intencional (ej. brazos formando hueco).
  - _DSC0084: Huecos internos detectados. Verificar si es intencional (ej. brazos formando hueco).
  - _DSC0119: Huecos internos detectados. Verificar si es intencional (ej. brazos formando hueco).
  - _DSC0143: Huecos internos detectados. Verificar si es intencional (ej. brazos formando hueco).
  - _DSC0281: Huecos internos detectados. Verificar si es intencional (ej. brazos formando hueco).
  ... y 3 más

Máscaras guardadas en: /content/drive/MyDrive/TFM/0_Imagenes_CVAT/ground_truth_masks

PROCESO COMPLETADO EXITOSAMENTE

Resumen de resultados:
  Total de máscaras ground truth generadas: 20
  Ubicación: 